In [39]:
import os
import random
import time

import PIL
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torch.utils.data import Subset

from torchvision import transforms

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from efficientnet_pytorch import EfficientNet
from sklearn.metrics import f1_score

from torch.optim.lr_scheduler import ReduceLROnPlateau
import argparse


In [2]:
class TrainDataset(Dataset):
    def __init__(self, data_frame: pd.DataFrame, root_dir: str, transform=None):
        self.data_frame = data_frame
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        sample = dict()
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = str(self.data_frame.iloc[idx]['id'])
        img_path = os.path.join(self.root_dir, img_name+'.jpg')
        image = PIL.Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        tag_name = self.data_frame.iloc[idx]['class']
        
        sample['image'] = image
        sample['class'] = tag_name
        
        return sample


In [3]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
                [0.485, 0.456, 0.406], 
                [0.229, 0.224, 0.225])

])

In [68]:
def train(model, train_loader, optimizer, criterion, epoch ):
    
    model.train()
    train_loss = 0.0
    
    for i, data in enumerate(train_loader):
        x = data['image']
        emotion = data['class']
        x = x.to(device)
        emotion = emotion.to(device)

        optimizer.zero_grad() 

        out = model(x)

        
        loss = criterion(out, emotion)

        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() / len(train_loader)
        #print(f'epoch is {epoch} | train_loss is {train_loss}')
    del x, emotion
    torch.cuda.empty_cache()
    return train_loss

def validation(model, criterion, valid_loader):
    
    model.eval()
    val_loss = 0.0
    label = []
    prediction = []
    
    with torch.no_grad():
        for i, data in enumerate(valid_loader):
            x = data['image']
            emotion = data['class']
            x = x.to(device)
            emotion = emotion.to(device)
            out = model(x)

            
            loss = criterion(out, emotion)
            
            pred = torch.argmax(out,dim=-1)
            
            val_loss += loss.item() / len(valid_loader)
            
            label = label + emotion.tolist()
            prediction = prediction + pred.detach().cpu().tolist()
        del x, emotion

    torch.cuda.empty_cache()
    val_score = f1_score(label, prediction, average='micro') 
    
    return val_loss, val_score

In [69]:
def train_model(args,num_epochs=60, cv_checkpoint=True, fine_tune=False,
                weight_file_name='weight_best.pt', **train_kwargs):
    # choose scheduler
    lr = args.learning_rate
    optimizer = optim.AdamW(model.parameters(),lr = lr,weight_decay = 1e-5)   
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.9)
    
    train_result = {}
    train_result['weight_file_name'] = weight_file_name
    best_epoch = -1
    best_score = 0.
    lrs = []
    score = []

    for epoch in range(num_epochs):
        
        start_time = time.time()

        train_loss = train(model, train_loader, optimizer,criterion,epoch)
        val_loss, val_score = validation(model, criterion, valid_loader)
        score.append(val_score)

        if cv_checkpoint:
            if val_score > best_score:
                best_score = val_score
                train_result['best_epoch'] = epoch + 1
                train_result['best_score'] = round(best_score, 5)
                
                torch.save(model.state_dict(), weight_file_name)
                print("Score is higher than last model .....Saving Model.....")
        else:
            if val_loss < best_loss:
                best_loss = val_loss
                train_result['best_epoch'] = epoch + 1
                train_result['best_loss'] = round(best_loss, 5)
                
                torch.save(model.state_dict(), weight_file_name)
        
        elapsed = time.time() - start_time
        now = time.localtime()
        lr = [_['lr'] for _ in optimizer.param_groups]
        print("Epoch {} - train_loss: {:.4f}  val_loss: {:.4f}  cv_score: {:.4f}  lr: {:.6f}  time: {:.0f}s nowtime: {},{}:{}.{}".format(
                epoch+1, train_loss, val_loss, val_score, lr[0], elapsed,now.tm_mday,now.tm_hour,now.tm_min,now.tm_sec))
      
        for param_group in optimizer.param_groups:
            lrs.append(param_group['lr'])
        
        # scheduler update
        if cv_checkpoint:
            scheduler.step(val_score)
        else:
            scheduler.step(val_loss)
       
     
    return train_result, lrs, score

In [70]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [79]:
parser = argparse.ArgumentParser()
args = parser.parse_args("")

df = pd.read_csv(f'../data/assignment/write.csv')
data_dir = '../data/assignment/preprocessed_train/'

SEED = 42
seed_everything(SEED)
args.batch_size = 128
args.learning_rate = 0.001
args.multi_parallel = True
args.num_epochs = 2000



In [ ]:

result_arr = []

batch_size = args.batch_size
#batch_size = 128*2
traindataset = TrainDataset(df,root_dir = data_dir,transform = train_transform)
train_set, val_set = torch.utils.data.random_split(traindataset, [int(len(traindataset)*0.8), len(traindataset)-int(len(traindataset)*0.8)])


train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

torch.cuda.empty_cache()

model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=5)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss(reduction='mean')
criterion = criterion.to(device)

train_kwargs = dict(
    train_loader=train_loader,
    valid_loader=valid_loader,
    model=model,
    criterion=criterion,
)

num_epochs = args.num_epochs
result, lrs, score = train_model(args,num_epochs=num_epochs, cv_checkpoint=True, fine_tune=False, **train_kwargs)
result_arr.append(result)
print(result)

# learning rate plot
plt.figure(figsize=(18,4))
plt.subplot(1,2,1)
plt.plot(lrs, 'b')
plt.xlabel('Epochs', fontsize=12, fontweight='bold')
plt.ylabel('Learning rate', fontsize=14, fontweight='bold')
plt.title(f' Learning rate schedule', fontsize=15, fontweight='bold')

x = [x for x in range(0, num_epochs, 10)]
y = [0.01, 0.005, 0.000001]
ylabel = ['1e-2', '1e-4', '1e-6']
plt.xticks(x)
plt.yticks(y, ylabel)

plt.subplot(1,2,2)
plt.plot(score, 'r')
plt.xlabel('Epochs', fontsize=12, fontweight='bold')
plt.ylabel('Valid score', fontsize=14, fontweight='bold')
plt.title(f'Fold  F1 Score', fontsize=15, fontweight='bold')

x = [x for x in range(0, num_epochs, 10)]

plt.show()
time.sleep(5)
plt.close()


print("train finish")

Loaded pretrained weights for efficientnet-b3
Score is higher than last model .....Saving Model.....
Epoch 1 - train_loss: 1.1456  val_loss: 1.9988  cv_score: 0.4410  lr: 0.001000  time: 15s nowtime: 23,17:54.34
Score is higher than last model .....Saving Model.....
Epoch 2 - train_loss: 0.4722  val_loss: 1.4607  cv_score: 0.6108  lr: 0.001000  time: 16s nowtime: 23,17:54.49
Score is higher than last model .....Saving Model.....
Epoch 3 - train_loss: 0.2192  val_loss: 1.2587  cv_score: 0.6910  lr: 0.001000  time: 15s nowtime: 23,17:55.5
Score is higher than last model .....Saving Model.....
Epoch 4 - train_loss: 0.1405  val_loss: 1.2399  cv_score: 0.7547  lr: 0.001000  time: 16s nowtime: 23,17:55.20
Epoch 5 - train_loss: 0.0989  val_loss: 1.4821  cv_score: 0.7382  lr: 0.001000  time: 15s nowtime: 23,17:55.35
Score is higher than last model .....Saving Model.....
Epoch 6 - train_loss: 0.0375  val_loss: 1.4063  cv_score: 0.7594  lr: 0.001000  time: 15s nowtime: 23,17:55.51
Score is highe

Epoch 71 - train_loss: 0.0150  val_loss: 1.6841  cv_score: 0.7807  lr: 0.000531  time: 15s nowtime: 23,18:12.29
Epoch 72 - train_loss: 0.0081  val_loss: 1.5826  cv_score: 0.7830  lr: 0.000531  time: 15s nowtime: 23,18:12.45
Epoch 73 - train_loss: 0.0061  val_loss: 1.5226  cv_score: 0.7925  lr: 0.000531  time: 15s nowtime: 23,18:13.0
Epoch 74 - train_loss: 0.0133  val_loss: 1.4567  cv_score: 0.7995  lr: 0.000531  time: 15s nowtime: 23,18:13.15
Epoch 75 - train_loss: 0.0339  val_loss: 2.0356  cv_score: 0.6934  lr: 0.000531  time: 15s nowtime: 23,18:13.31
Epoch 76 - train_loss: 0.0320  val_loss: 1.7239  cv_score: 0.7264  lr: 0.000531  time: 15s nowtime: 23,18:13.46
Epoch 77 - train_loss: 0.0083  val_loss: 1.3841  cv_score: 0.7783  lr: 0.000531  time: 15s nowtime: 23,18:14.1
Epoch 78 - train_loss: 0.0048  val_loss: 1.3489  cv_score: 0.7830  lr: 0.000531  time: 15s nowtime: 23,18:14.17
Epoch 79 - train_loss: 0.0017  val_loss: 1.3424  cv_score: 0.7830  lr: 0.000478  time: 15s nowtime: 23,18:

Epoch 143 - train_loss: 0.0022  val_loss: 1.5141  cv_score: 0.8066  lr: 0.000282  time: 15s nowtime: 23,18:30.56
Epoch 144 - train_loss: 0.0008  val_loss: 1.4822  cv_score: 0.8066  lr: 0.000282  time: 15s nowtime: 23,18:31.11
Epoch 145 - train_loss: 0.0003  val_loss: 1.4686  cv_score: 0.8042  lr: 0.000254  time: 15s nowtime: 23,18:31.27
Epoch 146 - train_loss: 0.0009  val_loss: 1.4424  cv_score: 0.8019  lr: 0.000254  time: 15s nowtime: 23,18:31.42
Epoch 147 - train_loss: 0.0002  val_loss: 1.4361  cv_score: 0.8066  lr: 0.000254  time: 15s nowtime: 23,18:31.57
Epoch 148 - train_loss: 0.0001  val_loss: 1.4439  cv_score: 0.8066  lr: 0.000254  time: 15s nowtime: 23,18:32.13
Epoch 149 - train_loss: 0.0001  val_loss: 1.4515  cv_score: 0.8090  lr: 0.000254  time: 15s nowtime: 23,18:32.28
Score is higher than last model .....Saving Model.....
Epoch 150 - train_loss: 0.0008  val_loss: 1.4485  cv_score: 0.8184  lr: 0.000254  time: 15s nowtime: 23,18:32.44
Epoch 151 - train_loss: 0.0001  val_loss:

Epoch 216 - train_loss: 0.0008  val_loss: 1.6355  cv_score: 0.7830  lr: 0.000135  time: 15s nowtime: 23,18:49.38
Epoch 217 - train_loss: 0.0001  val_loss: 1.6163  cv_score: 0.7948  lr: 0.000135  time: 15s nowtime: 23,18:49.54
Epoch 218 - train_loss: 0.0002  val_loss: 1.5996  cv_score: 0.7995  lr: 0.000135  time: 15s nowtime: 23,18:50.9
Epoch 219 - train_loss: 0.0018  val_loss: 1.5695  cv_score: 0.8090  lr: 0.000135  time: 15s nowtime: 23,18:50.24
Epoch 220 - train_loss: 0.0003  val_loss: 1.5676  cv_score: 0.8113  lr: 0.000135  time: 15s nowtime: 23,18:50.40
Epoch 221 - train_loss: 0.0006  val_loss: 1.5629  cv_score: 0.8090  lr: 0.000135  time: 15s nowtime: 23,18:50.55
Epoch 222 - train_loss: 0.0005  val_loss: 1.5422  cv_score: 0.8066  lr: 0.000122  time: 15s nowtime: 23,18:51.10
Epoch 223 - train_loss: 0.0002  val_loss: 1.5377  cv_score: 0.8066  lr: 0.000122  time: 15s nowtime: 23,18:51.26
Epoch 224 - train_loss: 0.0008  val_loss: 1.5310  cv_score: 0.8042  lr: 0.000122  time: 15s nowti

Epoch 289 - train_loss: 0.0000  val_loss: 1.4148  cv_score: 0.8042  lr: 0.000065  time: 15s nowtime: 23,19:8.20
Epoch 290 - train_loss: 0.0001  val_loss: 1.4134  cv_score: 0.8042  lr: 0.000065  time: 15s nowtime: 23,19:8.35
Epoch 291 - train_loss: 0.0001  val_loss: 1.4130  cv_score: 0.8042  lr: 0.000065  time: 15s nowtime: 23,19:8.50
Epoch 292 - train_loss: 0.0001  val_loss: 1.4159  cv_score: 0.8066  lr: 0.000065  time: 15s nowtime: 23,19:9.6
Epoch 293 - train_loss: 0.0025  val_loss: 1.4166  cv_score: 0.8090  lr: 0.000065  time: 15s nowtime: 23,19:9.21
Epoch 294 - train_loss: 0.0058  val_loss: 1.4387  cv_score: 0.8042  lr: 0.000065  time: 15s nowtime: 23,19:9.36
Epoch 295 - train_loss: 0.0021  val_loss: 1.4565  cv_score: 0.7995  lr: 0.000065  time: 15s nowtime: 23,19:9.52
Epoch 296 - train_loss: 0.0002  val_loss: 1.4572  cv_score: 0.7948  lr: 0.000065  time: 15s nowtime: 23,19:10.7
Epoch 297 - train_loss: 0.0015  val_loss: 1.4585  cv_score: 0.7948  lr: 0.000065  time: 15s nowtime: 23,1

Epoch 362 - train_loss: 0.0014  val_loss: 1.4499  cv_score: 0.8160  lr: 0.000034  time: 15s nowtime: 23,19:26.58
Epoch 363 - train_loss: 0.0000  val_loss: 1.4607  cv_score: 0.8160  lr: 0.000034  time: 15s nowtime: 23,19:27.14
Epoch 364 - train_loss: 0.0001  val_loss: 1.4652  cv_score: 0.8137  lr: 0.000034  time: 15s nowtime: 23,19:27.29
Epoch 365 - train_loss: 0.0005  val_loss: 1.4646  cv_score: 0.8137  lr: 0.000031  time: 15s nowtime: 23,19:27.44
Epoch 366 - train_loss: 0.0001  val_loss: 1.4667  cv_score: 0.8113  lr: 0.000031  time: 15s nowtime: 23,19:27.59
Epoch 367 - train_loss: 0.0000  val_loss: 1.4721  cv_score: 0.8113  lr: 0.000031  time: 15s nowtime: 23,19:28.15
Epoch 368 - train_loss: 0.0000  val_loss: 1.4732  cv_score: 0.8090  lr: 0.000031  time: 15s nowtime: 23,19:28.30
Epoch 369 - train_loss: 0.0000  val_loss: 1.4740  cv_score: 0.8090  lr: 0.000031  time: 16s nowtime: 23,19:28.46
Epoch 370 - train_loss: 0.0001  val_loss: 1.4730  cv_score: 0.8113  lr: 0.000031  time: 15s nowt

Epoch 435 - train_loss: 0.0000  val_loss: 1.4135  cv_score: 0.8160  lr: 0.000016  time: 15s nowtime: 23,19:45.39
Epoch 436 - train_loss: 0.0001  val_loss: 1.4121  cv_score: 0.8160  lr: 0.000016  time: 15s nowtime: 23,19:45.54
Epoch 437 - train_loss: 0.0000  val_loss: 1.4119  cv_score: 0.8160  lr: 0.000016  time: 16s nowtime: 23,19:46.10
Epoch 438 - train_loss: 0.0002  val_loss: 1.4123  cv_score: 0.8160  lr: 0.000016  time: 15s nowtime: 23,19:46.25
Epoch 439 - train_loss: 0.0023  val_loss: 1.4128  cv_score: 0.8137  lr: 0.000016  time: 15s nowtime: 23,19:46.41
Epoch 440 - train_loss: 0.0001  val_loss: 1.4234  cv_score: 0.8137  lr: 0.000016  time: 15s nowtime: 23,19:46.56
Epoch 441 - train_loss: 0.0227  val_loss: 1.4303  cv_score: 0.8184  lr: 0.000016  time: 15s nowtime: 23,19:47.11
Epoch 442 - train_loss: 0.0000  val_loss: 1.4372  cv_score: 0.8184  lr: 0.000015  time: 15s nowtime: 23,19:47.26
Epoch 443 - train_loss: 0.0000  val_loss: 1.4394  cv_score: 0.8208  lr: 0.000015  time: 15s nowt

Epoch 506 - train_loss: 0.0000  val_loss: 1.4148  cv_score: 0.8231  lr: 0.000009  time: 15s nowtime: 23,20:3.48
Epoch 507 - train_loss: 0.0001  val_loss: 1.4154  cv_score: 0.8231  lr: 0.000009  time: 15s nowtime: 23,20:4.4
Epoch 508 - train_loss: 0.0002  val_loss: 1.4145  cv_score: 0.8231  lr: 0.000008  time: 15s nowtime: 23,20:4.19
Epoch 509 - train_loss: 0.0001  val_loss: 1.4147  cv_score: 0.8184  lr: 0.000008  time: 15s nowtime: 23,20:4.34
Epoch 510 - train_loss: 0.0000  val_loss: 1.4152  cv_score: 0.8184  lr: 0.000008  time: 15s nowtime: 23,20:4.50
Epoch 511 - train_loss: 0.0000  val_loss: 1.4162  cv_score: 0.8184  lr: 0.000008  time: 15s nowtime: 23,20:5.5
Epoch 512 - train_loss: 0.0000  val_loss: 1.4145  cv_score: 0.8184  lr: 0.000008  time: 15s nowtime: 23,20:5.20
Epoch 513 - train_loss: 0.0015  val_loss: 1.4134  cv_score: 0.8184  lr: 0.000008  time: 15s nowtime: 23,20:5.36
Epoch 514 - train_loss: 0.0003  val_loss: 1.4103  cv_score: 0.8231  lr: 0.000008  time: 15s nowtime: 23,20

Epoch 579 - train_loss: 0.0001  val_loss: 1.3854  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:22.30
Epoch 580 - train_loss: 0.0000  val_loss: 1.3842  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:22.45
Epoch 581 - train_loss: 0.0000  val_loss: 1.3864  cv_score: 0.8184  lr: 0.000004  time: 16s nowtime: 23,20:23.1
Epoch 582 - train_loss: 0.0000  val_loss: 1.3855  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:23.16
Epoch 583 - train_loss: 0.0000  val_loss: 1.3859  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:23.32
Epoch 584 - train_loss: 0.0000  val_loss: 1.3851  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:23.47
Epoch 585 - train_loss: 0.0000  val_loss: 1.3853  cv_score: 0.8208  lr: 0.000004  time: 15s nowtime: 23,20:24.2
Epoch 586 - train_loss: 0.0016  val_loss: 1.3869  cv_score: 0.8184  lr: 0.000004  time: 15s nowtime: 23,20:24.18
Epoch 587 - train_loss: 0.0000  val_loss: 1.3819  cv_score: 0.8160  lr: 0.000004  time: 15s nowtim

Epoch 652 - train_loss: 0.0006  val_loss: 1.3754  cv_score: 0.8160  lr: 0.000002  time: 15s nowtime: 23,20:41.14
Epoch 653 - train_loss: 0.0001  val_loss: 1.3743  cv_score: 0.8184  lr: 0.000002  time: 15s nowtime: 23,20:41.30
Epoch 654 - train_loss: 0.0000  val_loss: 1.3755  cv_score: 0.8184  lr: 0.000002  time: 15s nowtime: 23,20:41.45
Epoch 655 - train_loss: 0.0000  val_loss: 1.3766  cv_score: 0.8184  lr: 0.000002  time: 15s nowtime: 23,20:42.0
Epoch 656 - train_loss: 0.0000  val_loss: 1.3778  cv_score: 0.8160  lr: 0.000002  time: 15s nowtime: 23,20:42.16
Epoch 657 - train_loss: 0.0000  val_loss: 1.3771  cv_score: 0.8160  lr: 0.000002  time: 15s nowtime: 23,20:42.31
Epoch 658 - train_loss: 0.0000  val_loss: 1.3768  cv_score: 0.8160  lr: 0.000002  time: 15s nowtime: 23,20:42.46
Epoch 659 - train_loss: 0.0000  val_loss: 1.3778  cv_score: 0.8160  lr: 0.000002  time: 15s nowtime: 23,20:43.2
Epoch 660 - train_loss: 0.0000  val_loss: 1.3777  cv_score: 0.8160  lr: 0.000002  time: 15s nowtim